# Dynamic scraper
#### Goal
As the title suggests, this jupyter notebook includes a data scraper. The goal is to crawl through https://www.portaldrazeb.cz and to collect actual data about auctions, auctioneers and a list of auction attributes, which we will subsequently use to filter the auctions when the data is processed.  
#### Problem
The problem is that the webpage has dynamic content and therefore it is not possible to easily extract the data we need since the "static" source code differs from the "dynamic" one. The website also does not provide API (it actually does, however, not for us and not for the purposes we need). 

#### Solution
We need to use proper methods to handle the dynamic content - our solution is the installation of package selenium and setting up a Google Chrome webdriver. We basically open the webpage, collect its source code and navigate between pages. Thanks to this package (and the webdriver which is also included in the GitHub repository) we manage to download all the data we need. More detailed description of particular methods can be found in the class docstring and in the comments.

In [1]:
#!pip install selenium

In [107]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
from tqdm import tqdm

In [108]:
url_auctions='https://www.portaldrazeb.cz/drazby/pripravovane' 
url_auctioneers = 'https://www.portaldrazeb.cz/drazebnici'

In [109]:
class DataDownloader:
    '''
    This class crawls through dynamic content of https://www.portaldrazeb.cz and collects following things:
    
            1) soup object for every auctioneer
            2) link to every auction + auction category (since the category is not within the auction page itself)
            3) list of all possible values from drop-down menu (auction categories, regions and districts)
    '''
    def __init__(self):
        # we initiate the lists for data within particular methods
        print('Downloader successfully initialized!')
        print(' ')
        print(DataDownloader.__doc__)
        
    def get_soups_of_auctioneers(self,link):
        '''
        Crawls through all pages of auctioneers and creates a soup object of everyone that is listed there right now.
        '''
        self.auctioneers_soups = []
        # initiating a webdriver
        driver = webdriver.Chrome('./chromedriver') 
        
        # opening the link in Chrome
        driver.get(link)  
        time.sleep(5) 
        
        # creating a soup object from the page source code
        soup = BeautifulSoup(driver.page_source, "html.parser")  
        
        # getting number of pages
        last_page = int(soup.find('div',{'class':'el-pagination'}).findAll('li',{'class':'number'})[-1].text) 
        
        # locating the element into which we write page number
        page_number = driver.find_element_by_css_selector('input[type="number"]')
        
        # looping through all pages and save the soups
        for page in tqdm(range(1,last_page+1)):
            # getting to a page (delete content, send number of the page, press Enter)
            page_number.send_keys(Keys.BACK_SPACE)
            page_number.send_keys(Keys.BACK_SPACE)
            page_number.send_keys(str(page)) 
            page_number.send_keys(Keys.RETURN)
            time.sleep(5)  

            # save soups of particular auctioneers
            html = driver.page_source   
            soup = BeautifulSoup(html, "html") # soup of current page  
            for i in soup.findAll('article'):
                self.auctioneers_soups.append(i) # extract all auctioneers
                
        # close the window and check soups
        driver.close()
        if len(self.auctioneers_soups)>50:
                print(f'Soup objects of auctioneers successfully downloaded! There are {len(self.auctioneers_soups)} of them right now.')
                
    def get_auction_links_and_categories(self,link):
        '''
        Crawls through all pages of auctions and from their source codes then collects link and category of every auction.
        
        '''
        self.auction_links_and_categories = []
        
        # initiating the webdriver and opening the link
        driver = webdriver.Chrome('./chromedriver')
        driver.get(url_auctions) 
        time.sleep(5) 
        
        # getting number of pages from a soup
        html = driver.page_source
        soup = BeautifulSoup(html, "html")
        last_page = int(soup.find('div',{'class':'el-pagination'}).findAll('li',{'class':'number'})[-1].text) # get number of pages
        
        # getting source codes from all pages and saving as soups
        page_number = driver.find_element_by_css_selector('input[type="number"]') # locating element into which we write page number
        auctions_pages_soups = []
        for page in tqdm(range(1,last_page+1)):
            # get to a page
            page_number.send_keys(Keys.BACK_SPACE)
            page_number.send_keys(Keys.BACK_SPACE)
            page_number.send_keys(str(page))
            page_number.send_keys(Keys.RETURN)
            time.sleep(5)
            # save soup object of the page
            html = driver.page_source 
            auctions_pages_soups.append(BeautifulSoup(html, "html"))

        for soup in auctions_pages_soups:
            for i in soup.findAll('article'):
                # extracting link
                auction = []
                auction.append(i.find('a')['href'])

                # extracting categories
                categ = soup.find('article').find('tbody').findAll('tr')[1].find('span').text.lstrip('/').split('/')
                auction.append(categ)
                self.auction_links_and_categories.append(auction) # saving the data
        
        # closing the window and checking whether something downloaded
        driver.close()
        if len(self.auction_links_and_categories)>200:
            print(f'Auction links and categories successfully downloaded! There are {len(self.auction_links_and_categories)} auctions right now.')
    def get_items_from_dropdown_menu(self,link):
        '''
        Downloads all auctions categories, regions and districts.
        '''
        self.categories = []
        self.regions_and_districts = []
        
        # initiating the webdriver and opening the link
        driver = webdriver.Chrome('./chromedriver')
        driver.get(url_auctions) 
        time.sleep(5) 
        
        # saving source code, extracting auction categories
        html = driver.page_source 
        soup = BeautifulSoup(html, "html")
        for categ in soup.findAll('ul',{'class':'el-scrollbar__view el-select-dropdown__list'})[0].findAll('span'):
            self.categories.append(categ.text)
            
        # extracting regions and districts
        for region in soup.findAll('ul',{'class':'el-scrollbar__view el-select-dropdown__list'})[1].findAll('ul',{'class':'el-select-group__wrap'}):
            aux = []
            for district in region.findAll('span'):
                aux.append(district.text)
            self.regions_and_districts.append([region.find('li').text,aux])
        
        # closing the window and checking whether everyhing downloaded
        driver.close()
        if (len(self.categories) > 5) & (len(self.regions_and_districts) == 14):
            print('Auction categories, regions and districts successfully downloaded!')

In [110]:
down = DataDownloader()

Downloader successfully initialized!
 

    This class crawls through dynamic content of https://www.portaldrazeb.cz and collects following things:
    
            1) soup object for every auctioneer
            2) link to every auction + auction category (since the category is not within the auction page itself)
            3) list of all possible values from drop-down menu (auction categories, regions and districts)
    


In [111]:
down.get_soups_of_auctioneers('https://www.portaldrazeb.cz/drazebnici') # takes approx. 1 minute

100%|██████████| 8/8 [00:41<00:00,  5.16s/it]


Soup objects of auctioneers successfully downloaded! There are 157 of them right now.


In [112]:
down.get_auction_links_and_categories('https://www.portaldrazeb.cz/drazby/pripravovane') # takes approx. 5 minutes

100%|██████████| 55/55 [04:46<00:00,  5.21s/it]


Auction links and categories successfully downloaded! There are 1092 auctions right now.


In [113]:
down.get_items_from_dropdown_menu('https://www.portaldrazeb.cz/drazby/pripravovane')

Auction categories, regions and districts successfully downloaded!


In [114]:
auctioneerslist=[]
for i in range(len(down.auctioneers_soups)):
    district=(down.auctioneers_soups)[i].find('a').text[7:]
    name = down.auctioneers_soups[i].findAll('li')[0].text.split(" ")
    address=down.auctioneers_soups[i].findAll('li')[1].text.replace(' ','').replace('\n','')
    postcode=down.auctioneers_soups[i].findAll('li')[2].text[:6].replace(' ','')
    town=down.auctioneers_soups[i].findAll('li')[2].text[7:]
    phone=down.auctioneers_soups[i].findAll('li')[3].text.replace(' ','')
    mail=down.auctioneers_soups[i].findAll('li')[4].text.replace(' ','')
    web=down.auctioneers_soups[i].findAll('li')[5].text.replace(' ','')
    execnumber=down.auctioneers_soups[i].findAll('li')[6].text[-3:]
    idnumber=down.auctioneers_soups[i].findAll('li')[7].text.replace(' ','').replace('\n','')[-8:]
    try:
        datacache=down.auctioneers_soups[i].findAll('li')[8].text.replace(' ','').replace('\n','')[-7:]
    except IndexError:
        datacache=np.nan
    auctioneer=[]
    auctioneer.extend([district,name[0],name[1],address,postcode,town,phone,mail,web,execnumber,idnumber,datacache])
    auctioneerslist.append(auctioneer)
auctioneerslist_in_pandas=pd.DataFrame(auctioneerslist)
auctioneerslist_in_pandas.columns=["District","Surname","Name","Address","Postcode","City","Phone number","E-mail","Web","Bailiff number","Identification number","Datacache"]

In [155]:
auctioneerslist_in_pandas=auctioneerslist_in_pandas[auctioneerslist_in_pandas.Surname != "Testovací"]
#The page includes a cell with nonexistent "test" bailiff, this gets rid of it.
auctioneerslist_in_pandas.to_csv('auctioneerslist.csv',encoding='UTF-8')
auctioneerslist_in_pandas


,District,Surname,Name,Address,Postcode,City,Phone number,E-mail,Web,Bailiff number,Identification number,Datacache
0,Benešov,Vychopňová,Iva,Jiráskova2042,25601,Benešov,311444834,info@eubenesov.cz,http://www.eubenesov.cz,170,72552565,fnd6rwr
1,Beroun,Dolanský,Pavel,Palackého31/2,26601,Beroun,311622657,podatelna@exekuceberoun.cz,http://www.exekuceberoun.cz,015,40658759,m8dg8bj
2,Blansko,Mynaříková,Marie,Masarykova1355/12,67801,Blansko,513035060,podatelna@exekuceblansko.cz,http://www.exekuceblansko.cz,210,06088953,w45ij7i
3,Břeclav,Cink,Libor,nám.T.G.Masaryka17,69002,Břeclav,519322060,info@exekutorbreclav.cz,http://www.exekutorbreclav.cz,123,71244484,qpeg79n
4,Břeclav,Petrošová,Marcela,17.listopadu2995/1a,69002,Břeclav,513033034,podatelna@exekucebreclav.cz,http://www.exekucebreclav.cz,160,72987898,r4dsxd7
...,...,...,...,...,...,...,...,...,...,...,...,...
152,Zlín,Jenerál,Marek,Krátká4447,76001,Zlín,577001640,podatelna@euzl.cz,http://www.euzl.cz,177,65850661,4e5pk2q
153,Zlín,Jarková,Jana,2.května2384,76001,Zlín,727881101,podatelna@euzlin.cz,http://www.euzlin.cz,207,05594936,xcz8zvx
154,Zlín,Brančík,Ctibor,nám.3.května1569,76502,Otrokovice,777661018,info@exekutor-zlin.cz,http://www.exekutor-zlin.cz,215,06624146,g9qvns3
155,Znojmo,Jaroš,Vojtěch,Rooseveltova2,66902,Znojmo,515222208,podatelna@euznojmo.cz,http://www.euznojmo.cz,133,75126516,3iyg8ih


In [157]:
auctionslist=[]
for i in range(50):#(len(down.auction_links_and_categories)), but for testing it is easier to work with just the beginning:
    req=requests.get(down.auction_links_and_categories[i][0])
    time.sleep(0.1)
    soup1=BeautifulSoup(req.text,"html.parser")
    auctiondata=soup1.findAll("div", {"class": "auction"})
    auctiondata=str(auctiondata[0])
    title=auctiondata[(auctiondata.index("item") + len("item")+12):]
    title=title[:title.index("category")-3]
    auctioneer=auctiondata[(auctiondata.index("auctioneer_office") + len("auctioneer_office")+30):]
    auctioneer=auctioneer[:auctioneer.index("addresses")-3].split(" ")
    number=auctiondata[(auctiondata.index("voluntary")-17):(auctiondata.index("voluntary")-3)]
    estimated_price=auctiondata[(auctiondata.index("estimated_price") + len("estimated_price")+2):]
    estimated_price=estimated_price[:estimated_price.index("item_price")-2]
    reserve_price=auctiondata[auctiondata.index("item_price")+12:]
    reserve_price=reserve_price[:reserve_price.index("minimal_bid")-2]
    auction=[]
    auction.extend([auctioneer[0],auctioneer[1],title,number,estimated_price,reserve_price])
    auctionslist.append(auction)
    print([i],sep=' ', end='', flush="true")#writes how many auctions it processed, just to ensure it actually does something
    #takes a shitload of time to scrape it all(20 to 30 minutes, probably)
    #Doesn`t recognize Czech letters
    

[0][1][2][3][4][5][6][7][8][9][10][11][12][13][14][15][16][17][18][19][20][21][22][23][24][25][26][27][28][29][30][31][32][33][34][35][36][37][38][39][40][41][42][43][44][45][46][47][48][49]

In [158]:
auctionslist_in_pandas = auctionslist_in_pandas=pd.DataFrame(auctionslist)
auctionslist_in_pandas.columns=["Surname","Name","Title of auction","Auction code","Estimated price","Reserve price"]
auctionslist_in_pandas.to_csv('auctionslist.csv',encoding='UTF-8')
auctionslist_in_pandas


,Surname,Name,Title of auction,Auction code,Estimated price,Reserve price
0,Kocinec,Jaroslav,1/2 pod\u00edl na R.D. \u010d.p. 446 v obci Lu...,EX00595/20-Luk,2280000,1520000
1,Preus,Pavel,Komer\u010dn\u00ed nemovitosti v centru Liberc...,X10410/19-1197,7020010,4680007
2,Preus,Pavel,Komer\u010dn\u00ed nemovitosti v centru Liberc...,X10410/19-1190,7020010,4680007
3,Pro\u0161ek,Ji\u0159\u00ed,"Hyundai Santa-Fe 2.2l nafta, rok v\u00fdroby 2006",EX11553/18-281,30000,10000
4,Voborn\u00edk,Tom\u00e1\u0161,"Rod. d\u016fm, pozemky a p\u0159\u00edslu\u016...",":""195DD3/18-10",7008000,2803200
5,Effenberger,Jakub,Pod\u00edl 1/2 pozemk\u016f v k.\u00fa. Nov\u0...,EX00236/19-587,482000,321330
6,Such\u00e1nek,Michal,Dra\u017eba pozemk\u016f se stavbou nezapsanou...,EX05761/18-198,225900,150600
7,Such\u00e1nek,Michal,Dra\u017eba rodinn\u00e9ho domu s p\u0159ilehl...,EX12831/10-416,1380000,920000
8,Koncz,David,Dra\u017eba souboru pen\u011b\u017eit\u00fdch ...,DD00003/21-005,1953665,1953665
9,Koncz,David,Dra\u017eba souboru pen\u011b\u017eit\u00fdch ...,D00004/21-0050,227407,227407
